In [12]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go




In [ ]:
# Use the U.S. ticker FOR LIFE 360 INC (LIF)
# FIND DAILY PRICE AND PERCENT CHANGE
ticker = yf.Ticker("LIF")

info = ticker.info
current_price = info.get("currentPrice")
previous_close = info.get("previousClose")
percent_change = ((current_price - previous_close) / previous_close) * 100

print(f"Current Price: ${current_price:.2f}")
print(f"Change: {percent_change:.2f}%")

Current Price: $97.68
Change: -0.19%


In [9]:
# Extract key metrics
metrics = {
    "Market Cap": info.get("marketCap"),
    "P/E Ratio": info.get("trailingPE"),
    "EPS (TTM)": info.get("trailingEps"),
    "Revenue (TTM)": info.get("totalRevenue"),
    "Last Updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

df_metrics = pd.DataFrame(metrics, index=["Life360"])
display(df_metrics)

,Market Cap,P/E Ratio,EPS (TTM),Revenue (TTM),Last Updated
Life360,7574680576,257.04343,0.38,427399008,2025-10-28 11:39:57


In [ ]:
#YTD PERFORMANCE DATA
# Get 1 year of daily data
hist = ticker.history(period="1y")

# 52-week stats
high_52 = hist["High"].max()
low_52 = hist["Low"].min()

# YTD performance
start_of_year = pd.Timestamp(datetime(datetime.now().year, 1, 1), tz='UTC')
ytd_data = hist.loc[hist.index >= pd.to_datetime(start_of_year)]
if not ytd_data.empty:
    start_price = ytd_data["Close"].iloc[0]
    current_price = ytd_data["Close"].iloc[-1]
    ytd_change = ((current_price - start_price) / start_price) * 100
else:
    ytd_change = None

df_ytd = pd.DataFrame({
    "Metric": ["52-Week High", "52-Week Low", "YTD Performance"],
    "Value": [
        f"${high_52:.2f}",
        f"${low_52:.2f}",
        f"{ytd_change:.2f}%" if ytd_change else "N/A"
    ]
})

display(df_ytd)



,Metric,Value
0,52-Week High,$112.54
1,52-Week Low,$29.62
2,YTD Performance,133.31%


In [ ]:
# Stock Price Chart
# Do we include this here or straight in Tableau?